In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
from utils_analysis import get_viewing_prop

In [2]:
obj_pos_upsample = np.load("../dataset/Nakano_etal_2010/preprocessed_data/obj_pos_upsample.npz")["data"]
gbvs_gaze_pos = np.load("../dataset/Nakano_etal_2010/preprocessed_data/gbvs_gaze_pos.npz", allow_pickle=True)

In [3]:
obj_pos_upsample.shape

(2, 18, 3883)

In [4]:
gbvs_labels = gbvs_gaze_pos['labels']
num_labels = len(gbvs_labels)
_, num_objects, num_sampling = obj_pos_upsample.shape

In [5]:
gbvs_gaze_pos_upsample = gbvs_gaze_pos['gaze_pos_upsample']

In [6]:
gaze_weight_dict = {}
gaze_weight_dict["labels"] = gbvs_labels
gaze_weight = np.zeros((num_labels, num_sampling, num_objects))
for i in range(num_labels):
    gaze_weight[i] = get_viewing_prop(gbvs_gaze_pos_upsample[i], 
                                      obj_pos_upsample[0], obj_pos_upsample[1])
gaze_weight_dict["all_data"] = gaze_weight

In [7]:
preproceeed_data_path = "../dataset/Nakano_etal_2010/preprocessed_data"
df = pd.read_csv(f"{preproceeed_data_path}/eyeclip_annotation.csv")
num_clip = df["clip_index"].max()

In [8]:
gaze_weight_dict["clip_data"] = {}
for clip_idx in tqdm(range(num_clip)):
    df_clip = df[df["clip_index"] == (clip_idx+1)].reset_index()
    sidx = df_clip["start_resampled_index"].unique()[0]
    eidx = df_clip["end_resampled_index"].unique()[0]
    subj_idx = df_clip["subj"].unique()
    num_faces = len(subj_idx)
    len_clip = eidx - sidx
    subj_slice = [df_clip[(df_clip["subj"] == s) & (df_clip["face"] == 1)].index.values for s in subj_idx]
    gaze_weight_dict["clip_data"][str(clip_idx)] = np.array([np.sum(gaze_weight[:, sidx:eidx, ss], axis=-1) for ss in subj_slice])

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
gaze_weight_dict.keys()

dict_keys(['labels', 'all_data', 'clip_data'])

In [9]:
np.savez_compressed("../dataset/Nakano_etal_2010/preprocessed_data/gaze_weight_gbvs.npz", 
                    **gaze_weight_dict)